# Alaska2 Baseline PyTorch

# Main Ideas

- 4 Classes
- GroupKFold splitting
- Class Balance
- Flips
- Label Smoothing
- EfficientNetB2
- ReduceLROnPlateau

# Dependencies

In [1]:
# !pip install efficientnet_pytorch
# !pip install albumentations
# !pip install sklearn
# !pip install pandas
# !pip install -U catalyst

In [2]:
# from catalyst.data.sampler import BalanceClassSampler
from glob import glob
from sklearn.model_selection import GroupKFold
import cv2
from skimage import io
import torch
from torch import nn
import os
from datetime import datetime
import time
import random
import cv2
import pandas as pd
import numpy as np
import albumentations as A
import matplotlib.pyplot as plt
from albumentations.pytorch.transforms import ToTensorV2
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
import sklearn

SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

In [3]:
os.environ["GIT_PYTHON_REFRESH"] = "quiet"
from catalyst.data.sampler import BalanceClassSampler

/opt/conda/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


# GroupKFold splitting

I think group splitting by image_name is really important for correct validation in this competition ;) 

In [4]:
# %%time

dataset = []

for label, kind in enumerate(['Cover', 'JMiPOD', 'JUNIWARD', 'UERD']):
    for path in glob('../input/data/Cover/*.jpg'):
        dataset.append({
            'kind': kind,
            'image_name': path.split('/')[-1],
            'label': label
        })

random.shuffle(dataset)
dataset = pd.DataFrame(dataset)

gkf = GroupKFold(n_splits=5)

dataset.loc[:, 'fold'] = -1
for fold_number, (train_index, val_index) in enumerate(gkf.split(X=dataset.index, y=dataset['label'], groups=dataset['image_name'])):
    dataset.loc[dataset.iloc[val_index].index, 'fold'] = fold_number

In any case original dataframe with splitting:

In [5]:
dataset.tail()

,kind,image_name,label,fold
299995,JMiPOD,44836.jpg,1,0
299996,JMiPOD,56953.jpg,1,3
299997,JMiPOD,73803.jpg,1,4
299998,Cover,13999.jpg,0,0
299999,Cover,62258.jpg,0,4


In [6]:
# dataset = pd.read_csv('../input/alaska2-public-baseline/groupkfold_by_shonenkov.csv')

# Simple Augs: Flips

In [7]:
def get_train_transforms():
    return A.Compose([
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.Resize(height=512, width=512, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.0)

def get_valid_transforms():
    return A.Compose([
            A.Resize(height=512, width=512, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.0)

# Dataset

In [8]:
DATA_ROOT_PATH = '../input/data'

def onehot(size, target):
    vec = torch.zeros(size, dtype=torch.float32)
    vec[target] = 1.
    return vec

class DatasetRetriever(Dataset):

    def __init__(self, kinds, image_names, labels, transforms=None):
        super().__init__()
        self.kinds = kinds
        self.image_names = image_names
        self.labels = labels
        self.transforms = transforms

    def __getitem__(self, index: int):
        kind, image_name, label = self.kinds[index], self.image_names[index], self.labels[index]
        image = cv2.imread(f'{DATA_ROOT_PATH}/{kind}/{image_name}', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        if self.transforms:
            sample = {'image': image}
            sample = self.transforms(**sample)
            image = sample['image']
            
        target = onehot(4, label)
        return image, target

    def __len__(self) -> int:
        return self.image_names.shape[0]

    def get_labels(self):
        return list(self.labels)

In [9]:
# fold_number = 0

# train_dataset = DatasetRetriever(
#     kinds=dataset[dataset['fold'] != fold_number].kind.values,
#     image_names=dataset[dataset['fold'] != fold_number].image_name.values,
#     labels=dataset[dataset['fold'] != fold_number].label.values,
#     transforms=get_train_transforms(),
# )

# validation_dataset = DatasetRetriever(
#     kinds=dataset[dataset['fold'] == fold_number].kind.values,
#     image_names=dataset[dataset['fold'] == fold_number].image_name.values,
#     labels=dataset[dataset['fold'] == fold_number].label.values,
#     transforms=get_valid_transforms(),
# )

# Metrics

In [10]:
from sklearn import metrics

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        
        
def alaska_weighted_auc(y_true, y_valid):
    """
    https://www.kaggle.com/anokas/weighted-auc-metric-updated
    """
    tpr_thresholds = [0.0, 0.4, 1.0]
    weights = [2, 1]

    fpr, tpr, thresholds = metrics.roc_curve(y_true, y_valid, pos_label=1)

    # size of subsets
    areas = np.array(tpr_thresholds[1:]) - np.array(tpr_thresholds[:-1])

    # The total area is normalized by the sum of weights such that the final weighted AUC is between 0 and 1.
    normalization = np.dot(areas, weights)

    competition_metric = 0
    for idx, weight in enumerate(weights):
        y_min = tpr_thresholds[idx]
        y_max = tpr_thresholds[idx + 1]
        mask = (y_min < tpr) & (tpr < y_max)
        # pdb.set_trace()
        if mask.sum() == 0:
            continue
        
        x_padding = np.linspace(fpr[mask][-1], 1, 100)

        x = np.concatenate([fpr[mask], x_padding])
        y = np.concatenate([tpr[mask], [y_max] * len(x_padding)])
        y = y - y_min  # normalize such that curve starts at y=0
        score = metrics.auc(x, y)
        submetric = score * weight
        best_subscore = (y_max - y_min) * weight
        competition_metric += submetric

    return competition_metric / normalization
        
class RocAucMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.y_true = np.array([0,1])
        self.y_pred = np.array([0.5,0.5])
        self.score = 0

    def update(self, y_true, y_pred):
        y_true = y_true.cpu().numpy().argmax(axis=1).clip(min=0, max=1).astype(int)
        y_pred = 1 - nn.functional.softmax(y_pred, dim=1).data.cpu().numpy()[:,0]
        self.y_true = np.hstack((self.y_true, y_true))
        self.y_pred = np.hstack((self.y_pred, y_pred))
        self.score = alaska_weighted_auc(self.y_true, self.y_pred)
    
    @property
    def avg(self):
        return self.score

# Label Smoothing

In [11]:
class LabelSmoothing(nn.Module):
    def __init__(self, smoothing = 0.05):
        super(LabelSmoothing, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing

    def forward(self, x, target):
        if self.training:
            x = x.float()
            target = target.float()
            logprobs = torch.nn.functional.log_softmax(x, dim = -1)

            nll_loss = -logprobs * target
            nll_loss = nll_loss.sum(-1)
    
            smooth_loss = -logprobs.mean(dim=-1)

            loss = self.confidence * nll_loss + self.smoothing * smooth_loss

            return loss.mean()
        else:
            return torch.nn.functional.cross_entropy(x, target)

# Fitter

In [12]:
import warnings

warnings.filterwarnings("ignore")
from pathlib import Path            

class Fitter:
    
    def __init__(self, model, device, config,fold):
        self.fold = fold
        self.config = config
        self.epoch = 0
        
        self.base_dir = '../models'
        self.log_path = f'{self.base_dir}/{self.fold}/log.txt'

        if not os.path.exists(f'{self.base_dir}/{self.fold}'):
            os.mkdir(f'{self.base_dir}/{self.fold}')
        if not os.path.exists(self.log_path):
            open(self.log_path, 'w+').close()

#         if not os.path.isfile(self.log_path):     
#             filename = Path(self.log_path)
#             filename.touch(exist_ok=True)

        self.best_summary_loss = 10**5

        self.model = model
        self.device = device
        
        param_optimizer = list(self.model.named_parameters())
        no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
            {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ] 

        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=config.lr)
        self.scheduler = config.SchedulerClass(self.optimizer, **config.scheduler_params)
        self.criterion = LabelSmoothing().to(self.device)
        self.log(f'Fitter prepared. Device is {self.device}')

    def fit(self, train_loader, validation_loader):
        
        oName = f'{self.base_dir}/{self.fold}'
        if not os.path.exists(oName):
            os.makedirs(oName)
            
        for e in range(self.config.n_epochs):
            if self.config.verbose:
                lr = self.optimizer.param_groups[0]['lr']
                timestamp = datetime.utcnow().isoformat()
                self.log(f'\n{timestamp}\nLR: {lr}')

            t = time.time()
            summary_loss, final_scores = self.train_one_epoch(train_loader)
            
            self.log(f'[RESULT]: Train. Epoch: {self.epoch}, summary_loss: {summary_loss.avg:.5f}, final_score: {final_scores.avg:.5f}, time: {(time.time() - t):.5f}')
            self.save(f'{self.base_dir}/{self.fold}/last-checkpoint.bin')

            t = time.time()
            summary_loss, final_scores = self.validation(validation_loader)

            self.log(f'[RESULT]: Val. Epoch: {self.epoch}, summary_loss: {summary_loss.avg:.5f}, final_score: {final_scores.avg:.5f}, time: {(time.time() - t):.5f}')
            if summary_loss.avg < self.best_summary_loss:
                self.best_summary_loss = summary_loss.avg
                self.model.eval()
                self.save(f'{self.base_dir}/{self.fold}/best-checkpoint-{str(self.epoch).zfill(3)}epoch.bin')
                for path in sorted(glob(f'{self.base_dir}/{self.fold}/best-checkpoint-*epoch.bin'))[:-3]:
                    os.remove(path)

            if self.config.validation_scheduler:
                self.scheduler.step(metrics=summary_loss.avg)

            self.epoch += 1

    def validation(self, val_loader):
        self.model.eval()
        summary_loss = AverageMeter()
        final_scores = RocAucMeter()
        t = time.time()
        for step, (images, targets) in enumerate(val_loader):
            if self.config.verbose:
                if step % self.config.verbose_step == 0:
                    print(
                        f'Val Step {step}/{len(val_loader)}, ' + \
                        f'summary_loss: {summary_loss.avg:.5f}, final_score: {final_scores.avg:.5f}, ' + \
                        f'time: {(time.time() - t):.5f}', end='\r'
                    )
            with torch.no_grad():
                targets = targets.to(self.device).float()
                batch_size = images.shape[0]
                images = images.to(self.device).float()
                outputs = self.model(images)
                loss = self.criterion(outputs, targets)
                final_scores.update(targets, outputs)
                summary_loss.update(loss.detach().item(), batch_size)

        return summary_loss, final_scores

    def train_one_epoch(self, train_loader):
        self.model.train()
        summary_loss = AverageMeter()
        final_scores = RocAucMeter()
        t = time.time()
        for step, (images, targets) in enumerate(train_loader):
            if self.config.verbose:
                if step % self.config.verbose_step == 0:
                    print(
                        f'Train Step {step}/{len(train_loader)}, ' + \
                        f'summary_loss: {summary_loss.avg:.5f}, final_score: {final_scores.avg:.5f}, ' + \
                        f'time: {(time.time() - t):.5f}', end='\r'
                    )
            
            targets = targets.to(self.device).float()
            images = images.to(self.device).float()
            batch_size = images.shape[0]

            self.optimizer.zero_grad()
            outputs = self.model(images)
            loss = self.criterion(outputs, targets)
            loss.backward()
            
            final_scores.update(targets, outputs)
            summary_loss.update(loss.detach().item(), batch_size)

            self.optimizer.step()

            if self.config.step_scheduler:
                self.scheduler.step()

        return summary_loss, final_scores
    
    def save(self, path):
        self.model.eval()
        torch.save({
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'scheduler_state_dict': self.scheduler.state_dict(),
            'best_summary_loss': self.best_summary_loss,
            'epoch': self.epoch,
        }, path)

    def load(self, path):
        checkpoint = torch.load(path)
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        self.scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        self.best_summary_loss = checkpoint['best_summary_loss']
        self.epoch = checkpoint['epoch'] + 1
        
    def log(self, message):
        if self.config.verbose:
            print(message)                        
        with open(self.log_path, 'a+') as logger:
            logger.write(f'{message}\n')

# EfficientNet

In [13]:
from efficientnet_pytorch import EfficientNet

def get_net(arch):
    net = EfficientNet.from_pretrained(arch)
    in_features  = net._fc.in_features
    net._fc = nn.Linear(in_features=in_features, out_features=4, bias=True)
    return net

# Config

In [14]:
class TrainGlobalConfig:
    num_workers = 4
    batch_size = 8
    n_epochs = 35
    lr = 0.001

    # -------------------
    verbose = True
    verbose_step = 1
    # -------------------

    # --------------------
    step_scheduler = False  # do scheduler.step after optimizer.step
    validation_scheduler = True  # do scheduler.step after validation stage loss
    
    SchedulerClass = torch.optim.lr_scheduler.ReduceLROnPlateau
    scheduler_params = dict(
        mode='min',
        factor=0.5,
        patience=1,
        verbose=False, 
        threshold=0.0001,
        threshold_mode='abs',
        cooldown=0, 
        min_lr=1e-8,
        eps=1e-08
    )
    # --------------------

# Class Balance "on fly" from [@CatalystTeam](https://github.com/catalyst-team/catalyst)

In [15]:
dataset.reset_index(drop=True,inplace=True)
dataset.head()

,kind,image_name,label,fold
0,JUNIWARD,34730.jpg,2,3
1,Cover,30243.jpg,0,1
2,JUNIWARD,24096.jpg,2,0
3,JMiPOD,07345.jpg,1,0
4,JMiPOD,38183.jpg,1,0


In [16]:

# _ = dataset[dataset['fold'] != 0].reset_index(drop=True)
# _.head()

In [17]:
from catalyst.data.sampler import BalanceClassSampler

def run_training(fold,arch='efficientnet-b6'):
    device = torch.device('cuda:0')

#     _ = dataset.reset_index()
    fold_number = fold

    a1 = dataset[dataset['fold'] != fold_number].reset_index(drop=True)
    a2 = dataset[dataset['fold'] == fold_number].reset_index(drop=True)
        
    train_dataset = DatasetRetriever(
        kinds=a1.kind.values,
        image_names=a1.image_name.values,
        labels=a1.label.values,
        transforms=get_train_transforms(),
    )

    validation_dataset = DatasetRetriever(
        kinds=a2.kind.values,
        image_names=a2.image_name.values,
        labels=a2.label.values,
        transforms=get_valid_transforms(),
    )

    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        sampler=BalanceClassSampler(labels=train_dataset.get_labels(), mode="downsampling"),
        batch_size=TrainGlobalConfig.batch_size,
        pin_memory=False,
        drop_last=True,
        num_workers=TrainGlobalConfig.num_workers,
    )
    val_loader = torch.utils.data.DataLoader(
        validation_dataset, 
        batch_size=TrainGlobalConfig.batch_size,
        num_workers=TrainGlobalConfig.num_workers,
        shuffle=False,
        sampler=SequentialSampler(validation_dataset),
        pin_memory=False,
    )

    net = get_net(arch).cuda()
    
    fitter = Fitter(model=net, device=device, config=TrainGlobalConfig,fold=fold_number)
    fitter.fit(train_loader, val_loader)

# Training

I have used 1xV100 for training model, in kaggle kernel it works also. You can make fork and check it, but I would like to share with you my logs

In [ ]:
arch = 'efficientnet-b4'
run_training(0,arch)
run_training(1,arch)
run_training(2,arch)
run_training(3,arch)
run_training(4,arch)

Loaded pretrained weights for efficientnet-b4
Fitter prepared. Device is cuda:0

2020-07-11T17:38:35.467340
LR: 0.001


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
# file = open('../input/alaska2-public-baseline/log.txt', 'r')
# for line in file.readlines():
#     print(line[:-1])
# file.close()

# Inference

In [ ]:
# checkpoint = torch.load('../input/alaska2-public-baseline/best-checkpoint-033epoch.bin')
# net.load_state_dict(checkpoint['model_state_dict']);
# net.eval();

In checkpoint you can find states for optimizer and scheduler if you need

In [ ]:
# checkpoint.keys()

In [ ]:
# class DatasetSubmissionRetriever(Dataset):

#     def __init__(self, image_names, transforms=None):
#         super().__init__()
#         self.image_names = image_names
#         self.transforms = transforms

#     def __getitem__(self, index: int):
#         image_name = self.image_names[index]
#         image = cv2.imread(f'{DATA_ROOT_PATH}/Test/{image_name}', cv2.IMREAD_COLOR)
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
#         image /= 255.0
#         if self.transforms:
#             sample = {'image': image}
#             sample = self.transforms(**sample)
#             image = sample['image']

#         return image_name, image

#     def __len__(self) -> int:
#         return self.image_names.shape[0]

In [ ]:
# dataset = DatasetSubmissionRetriever(
#     image_names=np.array([path.split('/')[-1] for path in glob('../input/alaska2-image-steganalysis/Test/*.jpg')]),
#     transforms=get_valid_transforms(),
# )


# data_loader = DataLoader(
#     dataset,
#     batch_size=8,
#     shuffle=False,
#     num_workers=2,
#     drop_last=False,
# )

In [ ]:
# %%time

# result = {'Id': [], 'Label': []}
# for step, (image_names, images) in enumerate(data_loader):
#     print(step, end='\r')
    
#     y_pred = net(images.cuda())
#     y_pred = 1 - nn.functional.softmax(y_pred, dim=1).data.cpu().numpy()[:,0]
    
#     result['Id'].extend(image_names)
#     result['Label'].extend(y_pred)

In [ ]:
# submission = pd.DataFrame(result)
# submission.to_csv('submission.csv', index=False)
# submission.head()

In [ ]:
# submission['Label'].hist(bins=100);

# Thank you for reading my kernel!